In [ ]:
# #############################################################################
# Importing necessary libraries

import numpy as np
import pandas as pd
import requests as requests
import json
from pathlib import Path
from m3inference import M3Inference
from collections import OrderedDict
import pprint
import urllib.request
import random
import re
import json
import os
from PIL import Image
import csv 
import sys
import string 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# Expanding the zip files
import zipfile as zf
files = zf.ZipFile("/home/sshuser2/IMDB_Images.zip", 'r')
files.extractall('/home/sshuser2/a')
files.close()

# Resize all the images in the same length and breath
def resize_im(image, hight, width):
    Image_a=Image.open(image)# Resizing the image
    resized_im = Image_a.resize((hight, width))
    return resized_im

# Convert all the images in the RGB formate
def Convert_format(resized_image):
    file_name=resized_image
    im = Image.open(resized_image)
    x,y=im.size
    if x!= y:
        #x,y size is not equal then it will resize the images 
        sqrWidth = np.ceil(np.sqrt(im.size[0]*im.size[1])).astype(int)
        im_resize = im.resize((sqrWidth, sqrWidth))
        pix = im_resize.load()
        Image.open(file_name).convert('RGB').save(file_name)
    else:
        Image.open(file_name).convert('RGB').save(file_name)
        
# Generate random string for JSON file
# def randomString(stringLength):
#     """Generate a random string of fixed length """
#     letters = string.ascii_lowercase
#     return ''.join(random.choice(letters) for i in range(stringLength))
name =""

# Generate random numbers
def randomNumber(Numberlength):
    for x in range(Numberlength):
        number= random.randint(1,10001)
    return number

# Running the M3 inference and getting the gender
def Mi_inference(Imagelocation,Imagename):
    config = Path(Imagelocation)
    value=randomNumber(10)
    Dummy_name=randomString(10)

    if config.is_file():                               # here we passing parameters in random both string and number
        data= {
            "description":"", 
            "id": value,
            "img_path": Imagelocation, 
            "lang": "en", 
            "name": name, 
            "screen_name": ""
        }
        Json_file='/home/sshuser2/M3/data.jsonl'        # Path to create the Json file
        with open(Json_file, 'w') as json_file:
            json.dump(data, json_file) 
        try:                                            # Here Based on values in output will select the geneder
            m3 = M3Inference()                          # see docstring for details
            pred = m3.infer(Json_file)                  # also see docstring for details
            disc=pprint.pprint(pred)
            main=dict(pred).get(value)
            main_gender=main.get('gender')
            main_gender_female=main_gender.get('female')
            main_gender_male=main_gender.get('male')
            if main_gender_female > main_gender_male :
                Gender='F'
            elif main_gender_female < main_gender_male :
                Gender='M'
            else:
                Gender='N'
        except urllib.error.HTTPError as e:
            print (e)
    else:
        Gender='NI'
    return Gender

# Running the imageson M3 and getting the output
with open('/home/sshuser2/imdb.csv', 'w', newline='') as myfile:             # Here output written in the file
    wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
    wr.writerow(['Imagename','G_Gender','M_Gender'])

    for Imagename in os.listdir('/home/sshuser2/a/'):                        # Get the images from directory
        try:
            Image_location="/home/sshuser2/a/"+Imagename                     # Get the image loacation of each image
            resized_image=resize_im(Image_location,200,200)
            Convert_format(Image_location)
            Mi_gender=Mi_inference(Image_location,Imagename)
            wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
            wr.writerow([Imagename,'F',Mi_gender])
        except:
            print("Oops!",sys.exc_info()[0],"occured.")
            print()
